In [1]:
from pikov import pikov
gamekitty = pikov.Pikov.open('gamekitty.pikov')

In [3]:
gamekitty._preview_clip(gamekitty.start_frame)

TypeError: unsupported operand type(s) for /: 'list' and 'int'

MultiClip(frames=(Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(id='1'), Frame(i

In [4]:
clip = gamekitty.get_clip('stand_waggle')
clip

In [5]:
# Show the mirrored version of the same frames.
# gamekitty.get_clip('stand_waggle_mirror')

In [6]:
clip.frames[1]

In [7]:
clip.frames[0].image

In [8]:
gamekitty.get_image('md5-9b084a2f4cc7a7f940fac47ebe710c45')

In [9]:
sit = gamekitty.get_clip('sit')
sit_to_stand = gamekitty.get_clip('sit_to_stand')
sit_paw = gamekitty.get_clip('sit_paw')
stand_to_sit = gamekitty.get_clip('stand_to_sit')
sit + sit_to_stand + clip + clip + stand_to_sit + sit_paw